In [1]:
import os
import base64
import requests
import json
import glob

# Define the parent directory to search for Excel files
parent_dir = r"C:\Users\Sachin.Saurav\OneDrive - insidemedia.net\Multiple_Documents"

# Directory to save processed output files
output_dir = r"C:\Users\Sachin.Saurav\OneDrive - insidemedia.net\Multiple_Documents\output_files"
os.makedirs(output_dir, exist_ok=True)

# Define the API endpoint URL
api_url = 'http://127.0.0.1:5000/process_excelfile'

# Function to encode a file to Base64
def encode_file_to_base64(file_path):
  with open(file_path, 'rb') as f:
      file_content = f.read()
  return base64.b64encode(file_content).decode('utf-8')

# Function to save the Base64-encoded Excel content to a file
def save_excel_file(encoded_data, output_file_path):
  with open(output_file_path, 'wb') as f:
      f.write(base64.b64decode(encoded_data))

# Collect all .xlsx files from the parent directory and its subdirectories
input_files = glob.glob(os.path.join(parent_dir, "**", "*.xlsx"), recursive=True)

if not input_files:
  print("No Excel files found in the specified directory or its subdirectories.")
else:
  # Prepare the JSON payload for the API
  files_payload = []
  for file_path in input_files:
      encoded_content = encode_file_to_base64(file_path)
      files_payload.append({
          "xlsx-name": os.path.basename(file_path),
          "attach-body": {
              "contentBytes": encoded_content
          }
      })

  payload = {"files": files_payload}

  # Set the headers for the POST request
  headers = {
      "Content-Type": "application/json"
  }

  # Send the POST request to the API
  response = requests.post(api_url, json=payload, headers=headers)

  # Print the response status and content
  print("Status Code:", response.status_code)
  try:
      response_json = response.json()
      print("Response Content:", json.dumps(response_json, indent=4))

      # Save the processed files from the response
      for result in response_json:
          if result['status'] == 'success':
              output_file_name = result.get('output_filename', f"Output_{result['filename']}")
              output_file_path = os.path.join(output_dir, output_file_name)
              save_excel_file(result['data'], output_file_path)
              print(f"Processed file saved to: {output_file_path}")
          else:
              print(f"Error processing file '{result['filename']}': {result['message']}")
              if 'details' in result:
                  print("Details:", result['details'])

  except Exception as e:
      print("An error occurred while processing the response:", str(e))



Status Code: 200
Response Content: [
    {
        "filename": "InputFile.xlsx",
        "message": "expected str, bytes or os.PathLike object, not NoneType",
        "status": "error"
    },
    {
        "filename": "SunMusic.xlsx",
        "message": "expected str, bytes or os.PathLike object, not NoneType",
        "status": "error"
    },
    {
        "filename": "zee.xlsx",
        "message": "expected str, bytes or os.PathLike object, not NoneType",
        "status": "error"
    }
]
Error processing file 'InputFile.xlsx': expected str, bytes or os.PathLike object, not NoneType
Error processing file 'SunMusic.xlsx': expected str, bytes or os.PathLike object, not NoneType
Error processing file 'zee.xlsx': expected str, bytes or os.PathLike object, not NoneType
